In [1]:
import os
import sys
from sqlalchemy import create_engine
from sqlalchemy import Column, INTEGER, TEXT, NUMERIC, REAL, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from urllib.parse import unquote

In [2]:
Base = declarative_base()

class Feature(Base):
    __tablename__ = 'features'
    featureid = Column(INTEGER, primary_key = True)
    seqid = Column(TEXT)
    source = Column(TEXT)
    type = Column(TEXT)
    start = Column(INTEGER)
    end = Column(INTEGER)
    score = Column(REAL)
    strand = Column(TEXT)
    phase = Column(TEXT)

    def __repr__(self):
        return f'''<Feature(featureid = {self.featureid}, seqid = {self.seqid}, source = {self.source}, type = {self.type},
                start = {self.start}, end = {self.end}, score = {self.score}, strand = {self.strand}, phase = {self.phase})>'''

class Attribute(Base):
    __tablename__ = 'attributes'

    id = Column(INTEGER, primary_key = True)
    featureid = Column(TEXT, ForeignKey('features.featureid'))
    tag = Column(TEXT)
    value = Column(TEXT)

    def __repr__(self):
            return f'<Attribute(featureid = {self.featureid}, tag = {self.tag}, value = {self.value})>'

In [37]:
def db_init(name):
    global Base, engine, Feature, Attribute, session
    db_name = name  
    if os.path.isfile(db_name):
        os.remove(db_name)
    engine = create_engine(f'sqlite:///{db_name}', echo=True)
    
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()

In [143]:
def attributes(index, attr_string):
    for key_vals in attr_string.split(';'):
        key, val = key_vals.split('=')
        val = val.split(',')
        for v in val:
            att = Attribute(**{'featureid' :index, 'tag':key, 'value':unquote(v)})
            session.add(att)

In [144]:
def features(index, gff_entry):
    entry = gff_entry[:]
    entry.insert(0, index)
    for i, e in enumerate(entry):
        try:
            if e == '.':
                entry[i] = None
                continue
            entry[i] = int(e)
        except ValueError:
            pass
    kwargs = dict(zip(['featureid', 'seqid', 'source', 'type', 'start', 
                       'end','score', 'strand', 'phase'], entry))
    return Feature(**kwargs)

In [ ]:
def main():
    try:
        path, db = sys.argv[1:3]
    except Exception:
        path = 'Caenorhabditis_elegans.WBcel235.91.gff3'
        db = os.path.join('.', 'c_elegans.sqlite')
    with open(path) as gff:
        db_init(db)
        i = 0
        for entry in gff:
            if entry.startswith('#'):
                continue
            else:
                i += 1
                entry = entry.split('\t')
                features(i, entry[:-1])
                attributes(i, entry[-1])

if __name__ == '__main__':
    main()

In [145]:
db_init(os.path.join('.', 'c_elegans.sqlite'))

2018-03-21 13:32:22,018 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-21 13:32:22,019 INFO sqlalchemy.engine.base.Engine ()
2018-03-21 13:32:22,020 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-21 13:32:22,021 INFO sqlalchemy.engine.base.Engine ()
2018-03-21 13:32:22,022 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("features")
2018-03-21 13:32:22,022 INFO sqlalchemy.engine.base.Engine ()
2018-03-21 13:32:22,023 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("attributes")
2018-03-21 13:32:22,024 INFO sqlalchemy.engine.base.Engine ()
2018-03-21 13:32:22,025 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE features (
	featureid INTEGER NOT NULL, 
	seqid TEXT, 
	source TEXT, 
	type TEXT, 
	start INTEGER, 
	"end" INTEGER, 
	score REAL, 
	strand TEXT, 
	phase TEXT, 
	PRIMARY KEY (featureid)
)


2018-03-21 13:32:22,026 INFO sqlalchemy.engine.base.Engine ()
2018-03-21

In [146]:
feat = features(i, entry[:-1])
session.add(feat)

In [147]:
session.commit()

2018-03-21 13:32:32,027 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-21 13:32:32,029 INFO sqlalchemy.engine.base.Engine INSERT INTO features (featureid, seqid, source, type, start, "end", score, strand, phase) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2018-03-21 13:32:32,030 INFO sqlalchemy.engine.base.Engine (11, 'I', 'WormBase', 'CDS', 5195, 5296, None, '-', 0)
2018-03-21 13:32:32,032 INFO sqlalchemy.engine.base.Engine COMMIT


In [21]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///c_elegans.sqlite', echo = True)
Session = sessionmaker(bind=engine)
session = Session()

In [38]:
print(session.query(Feature).count())

2018-03-21 14:37:45,238 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT features.featureid AS features_featureid, features.seqid AS features_seqid, features.source AS features_source, features.type AS features_type, features.start AS features_start, features."end" AS features_end, features.score AS features_score, features.strand AS features_strand, features.phase AS features_phase 
FROM features) AS anon_1
2018-03-21 14:37:45,239 INFO sqlalchemy.engine.base.Engine ()
659621


In [42]:
q = session.query(Attribute)

In [43]:
q.select_entity_from()

TypeError: select_entity_from() missing 1 required positional argument: 'from_obj'

In [44]:
engine.table_names()

2018-03-21 16:27:35,172 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-03-21 16:27:35,173 INFO sqlalchemy.engine.base.Engine ()


['attributes', 'features']

In [1]:
from sqlite3 import connect
from urllib.parse import unquote

In [47]:
conn = connect('c_elegans.sqlite')
curs = conn.cursor()

In [48]:
def make_table(cursor):
    features = r'''CREATE TABLE features (
        featureid INTEGER PRIMARY KEY,
        seqid TEXT,
        source TEXT,
        type TEXT,
        "start" INTEGER NOT NULL,
        "end" INTEGER NOT NULL,
        score REAL,
        strand TEXT,
        phase TEXT 
    );'''
    
    attributes = r'''CREATE TABLE attributes (
        featureid INTEGER NOT NULL,
        tag TEXT,
        "value" TEXT,
        FOREIGN KEY (featureid) REFERENCES features(featureid)
    );'''
    
    cursor.execute('DROP TABLE IF EXISTS features')
    cursor.execute('DROP TABLE IF EXISTS attributes')
    
    cursor.execute(features)
    cursor.execute(attributes)

In [49]:
make_table(curs)

In [50]:
def insert_features(cursor, featureid, features):
    statement = 'INSERT INTO features VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)'
    cursor.execute(statement, [featureid] + features)

In [51]:
def insert_attributes(cursor, featureid, attributes):
    statement = 'INSERT INTO attributes VALUES(?,?,?)'
    for tv in attributes.split(';'):
        tag, value = tv.split('=')
        value = value.split(',')
        for v in value:
            cursor.execute(statement, [featureid, tag, unquote(v)])

In [52]:
with open('Caenorhabditis_elegans.WBcel235.91.gff3') as gff:
    i = 0
    for line in gff:
        if line.startswith('#'): continue
        i += 1
        line = line.strip().split('\t')
        insert_features(curs, i, line[:-1])
        insert_attributes(curs, i, line[-1])
conn.commit()

In [46]:
curs.close()
conn.close()

In [53]:
curs.execute('CREATE INDEX features_seqid_idx ON features (seqid);')

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import func
from gffmappers import *

In [19]:
engine = create_engine('sqlite:///c_elegans_core.sqlite', echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [21]:
session.query(Attribute).limit(11).all()

[<Attribute(featureid = 1, tag = ID, value = chromosome:I)>,
 <Attribute(featureid = 1, tag = Alias, value = BX284601.5)>,
 <Attribute(featureid = 1, tag = Alias, value = NC_003279.8)>,
 <Attribute(featureid = 2, tag = ID, value = gene:WBGene00023193)>,
 <Attribute(featureid = 2, tag = Name, value = Y74C9A.6)>,
 <Attribute(featureid = 2, tag = biotype, value = snoRNA)>,
 <Attribute(featureid = 2, tag = gene_id, value = WBGene00023193)>,
 <Attribute(featureid = 2, tag = logic_name, value = wormbase_non_coding)>,
 <Attribute(featureid = 3, tag = ID, value = transcript:Y74C9A.6)>,
 <Attribute(featureid = 3, tag = Parent, value = gene:WBGene00023193)>,
 <Attribute(featureid = 3, tag = Name, value = Y74C9A.6)>]

In [16]:
session.query(Feature).count()

659621

In [17]:
session.query(Attribute).count()

3436229

In [18]:
session.close()